### Домашнее задание по ВКонтакте

Шаврина Татьяна

Сообщество - "badcomedian"
https://vk.com/badcomedian

1. Написать программу, которая обращается к открытому (то есть такому, чтобы для просмотра контента не нужна была авторизация) сообществу VK.com, выкачивает посты со стены и комментарии к ним. Выберите сообщество поживее, чтобы там было много постов и много комментариев к ним. Обязательное условие: программа должна уметь скачивать со стены больше, чем 100 постов, и больше, чем 100 комментариев к посту (если их действительно больше 100).
Чтобы не было скучно, посчитайте питоном и сложите в репозиторий в виде csv (или tsv) -таблиц длину каждого поста и каждого комментария в словах (для постов и комментариев отдельные таблицы), а потом с помощью того, что вы изучили на курсе анализа данных (то есть используя R) выведите некоторые графики описательной статистики по этому поводу. Ну, например, гистограмму длины постов и гистограмму длины комментариев. Графики, как и таблицы, на которых они будут построены, нужно положить в папку с домашним заданием, а также встроить в README.md-файл папки с домашним заданием. Как встраивать картинки в маркдаун, можно посмотреть, например, тут. Сами скачанные тексты прикладывать не нужно.

2. Усовершенствовать программу из первой части задания так, чтобы она не просто выкачивала посты и комментарии, но и смотрела в профиль пользователя, который написал пост или сделал комментарий, и узнавала о нём социолингвистическую информацию: возраст и город (если они указаны). Для города достаточно id (то есть название узнавать не обязательно, хотя это можно сделать средствами API), а возраст нужно уметь вычислять. 
Для каждого возраста нужно вычислить среднюю длину поста и комментария, нарисовать график (средствами R), отражающий эту информацию. Исходные данные в виде таблицы положить в репозиторий, а графики встроить в README.md-файл папки с домашним заданием. Сами скачанные тексты выкладывать не нужно.

In [1]:
import json
import urllib.request
import requests
import re

In [2]:
def GetUser(userid):
    url = u'https://api.vk.com/method/users.get?'
    url = url + "user_ids=" +  str(userid) + "&fields=bdate,city"
    res = urllib.request.urlopen(url).read().decode('utf-8')
    info = json.loads(res)
    
    if 'bdate' in info['response'][0]:
        date = info['response'][0]['bdate']
        if len(date.split("."))<3:
            age = "None"
        else:
            if int(date.split(".")[1])<4:
                age = str(2017-int(date.split(".")[2]))
            else:
                age = str(2016-int(date.split(".")[2]))
        
    else:
        age = "None"
    city = str(info['response'][0]['city'])

    return[age,city]

In [3]:
def Counter(line):
    length = str(len(line.split(" ")))
    return(length)

In [11]:
def GetComments(postid):
    commentstable = open('/home/mi_air/Desktop/progr/bc_wall_comments.csv', 'a', encoding='utf-8')
    url = u'https://api.vk.com/method/wall.getComments?'
    i=0
    while i<1000:
        url = url + "owner_id=-25557243&post_id=" + str(postid) +"&offset=" + str(i) + "&count=100"
        res = urllib.request.urlopen(url).read().decode('utf-8')
        wall = json.loads(res)
        for j in range(1,100):
            try:
                lst = []
                groupid = "-25557243"
                commid = str(wall['response'][j]['cid'])
                text  = wall['response'][j]['text']
                length = Counter(text)
                authorid = str(wall['response'][j]['from_id'])
                age,city = GetUser(authorid)
                lst.append(groupid)
                lst.append(postid)
                lst.append(commid)
                lst.append(length)
                lst.append(authorid)
                lst.append(str(age))
                lst.append(str(city))
                stroka = "\t".join(lst)
                commentstable.write(stroka + "\n")
                print(stroka)
            except:
                pass
        i+=100
    commentstable.close()

In [6]:
def GetPosts(groupid):
    poststable = open('/home/mi_air/Desktop/progr/bc_wall_posts.csv', 'a', encoding='utf-8')
    url = u'https://api.vk.com/method/wall.get?'

    i=0
    while i<1000:

        url = url + "owner_id=" +str(groupid) + "&offset=" + str(i) + "&count=100"
        res = urllib.request.urlopen(url).read().decode('utf-8')
        wall = json.loads(res)
        for j in range(1,100):
            groupid = "-25557243"
            postid = str(wall['response'][j]['id'])            
            text  = wall['response'][j]['text']
            if "signer_id" in wall['response'][j]:
                authorid = str(wall['response'][j]['signer_id'])
                age,city = GetUser(authorid)
            else:
                authorid = "None"
                age,city = "None","None"
            comments = wall['response'][j]['comments']['count']
            if comments>0:
                GetComments(postid)
            likes = str(wall['response'][j]['likes']['count'])
            length = Counter(text)
            stroka = "\t".join(lst)
            poststable.write(stroka + "\n") 
            print(stroka)

        i+=100
    poststable.close()

In [6]:
#main

commentstable = open('/home/mi_air/Desktop/progr/bc_wall_comments.csv', 'w', encoding='utf-8')
poststable = open('/home/mi_air/Desktop/progr/bc_wall_posts.csv', 'w', encoding='utf-8')
head = "group id" + "\t" + "post id" + "author id" + "\t" + "age" + "\t" + "city id"+ "\t" + "comments" + "\t" + "likes"  + "\t" + "text length" 
poststable.write(head + "\n")
head2 = "group id" + "\t" + "post id" + "\t" + "comment id" + "\t" + "text length" +  "\t" + "author id" + "\t" + "age" + "\t" + "city id"
commentstable.write(head2 + "\n")
commentstable.close()
poststable.close()




In [9]:

poststable = open('/home/mi_air/Desktop/progr/bc_wall_posts.csv', 'a', encoding='utf-8')
url = u'https://api.vk.com/method/wall.get?'

i=0
while i<2000:

    url = url + "owner_id=-25557243"  + "&offset=" + str(i) + "&count=100"
    res = urllib.request.urlopen(url).read().decode('utf-8')
    wall = json.loads(res)
    for j in range(1,100):
        lst = []
        groupid = "-25557243"
        lst.append(groupid)
        postid = str(wall['response'][j]['id'])
        lst.append(postid )
        text  = wall['response'][j]['text']
        if "signer_id" in wall['response'][j]:
            authorid = str(wall['response'][j]['signer_id'])
            age,city = GetUser(authorid)
        else:
            authorid = "None"
            age,city = "None","None"
        lst.append(authorid)
        lst.append(age)
        lst.append(city)
        comments = str(wall['response'][j]['comments']['count'])
        lst.append(comments)
        if int(comments)>0:
            GetComments(postid)
        likes = str(wall['response'][j]['likes']['count'])
        lst.append(likes)
        length = Counter(text)
        lst.append(length)
        stroka = "\t".join(lst)
        poststable.write(stroka+ "\n") 
        print(stroka)

    i+=100
    #poststable.close()